In [1]:
import ipyleaflet as ipl
import ipywidgets as widgets
import toml
import json
import requests
import collections
import arrow

from IPython.core.display import HTML

# See readme.md for credentials.toml formatting
cred = toml.load('credentials.toml')
appid = cred['api']['here']['appid']
appcode = cred['api']['here']['appcode']

def open_file(library_only=True):
    
    try:
        with open('library.geo.json', 'r') as l:
            collection = json.load(l)

        if library_only:
            library = collection['features']
            return library
        else:
            return collection

    except:
        return None

library = open_file(library_only=True)
collection = open_file(library_only=False)

def address_to_coord(address_string):
    """Return a tuple of the Lat/Long of the address."""
    raw_address = '+'.join(address_string.replace(',', ' ').split(' '))
    formatted_address = raw_address.replace('++', '+')

    url = "https://geocoder.api.here.com/6.2/geocode.json" +\
        "?app_id=" +\
        appid +\
        "&app_code=" +\
        appcode +\
        "&searchtext=" +\
        formatted_address

    response = requests.get(url)

    data = response.json()
    lat_long = data['Response']['View'][0]['Result'][0]['Location']['NavigationPosition'][0]
    point = coord_to_tuple(lat_long)
        
    return point



def coord_to_tuple(coordinates):
    """Transform coordinates to a 2 tuple."""
    if type(coordinates) == str:
        center = tuple(map(float, coordinates.split(',')))

    elif type(coordinates) == dict:
        center = tuple(coordinates.values())

    elif type(coordinates) == tuple:
        center = coordinates

    elif (type(coordinates) == list) and (len(coordinates) == 2):
        center = tuple(coordinates)

    else:
        print('invalid coordinate type')
        center = None

    return center

def coord_to_string(coordinates):
    """Transform coordinates into a string for API"""
    if type(coordinates) == tuple:
        string = ','.join(list(map(str, coordinates)))
        
    elif type(coordinates) == dict:
        string = ','.join(list(map(str, coordinates.values())))
    
    elif (type(coordinates) == list) and (len(coordinates) == 2):
        string = ','.join(list(map(str, coordinates)))
        
    elif type(coordinates) == str:
        string = coordinates
        
    else:
        print('invalid coordinate type')
        string = None        

    return string


def item_to_string(new_item):
    output_string = "<b>Coordinates:</b>  " + str(new_item['geometry']['coordinates']) + "</br>"

    for key,value in new_item['properties'].items():

        output_string = output_string + "<b>" + key.title() + ":</b>  " + str(value) + "</br>"
    
    return output_string

def saved_message(new_item):
    title = """<h2>New Submission Accepted</h2></br>
                This item will become visible after it has passed through our curation and moderation process.</br></br>"""
    
    body = item_to_string(new_item)
    
    raw_html = title + body
    
    return HTML(raw_html)


def save_library(library):

    collection = {"type":"FeatureCollection", 
                  "metadata":{"art_types":["Sculpture",
                                           "Mural",
                                           "Painting",
                                           "Mosaic",
                                           "Statue",
                                           "Fountain",
                                           "Building",
                                           "Installation Art",
                                           "Mechanical Art",
                                           "Folk Art",
                                           "Experience Art",
                                           "Other",
                                           
                                          ],
                              "warnings":["Adult Themes",
                                           "Daylight Only",
                                           "Private Work",
                                           "Controversial",
                                           "Distance Viewing Only",
                                           "Poor Accessibility",
                                           "No Nearby Parking",
                                           "High Crime Area",
                                           "None",
                                         
                                          ],
                              "last_updated": str(arrow.now())
                             }, 
                  
                  "features":library}
    
    
    
    try:
        with open('library.geo.json', 'w') as w:
            json.dump(collection, w)
            
        return True

    except:
        return False

    
    
def json_item(art_type="",
              name="",
              coordinates=None,
              geo_type='Point',
              address="",
              description="",
              artist="Unknown",
              hazards="",
              approved=False,):
    
    
    coordinates = marker.location
    
    item = {
              "type": "Feature",
              "geometry": {
                "type": geo_type,
                "coordinates": coordinates
              },
              "properties": {
                "name": name,
                  "art_type": art_type,
                  "address": address,
                  "description": description,
                  "artist":artist,
                  "hazards":hazards,
                  "approved": approved
              }
            }
    return item

def save_new_item(x):
    
    out.clear_output(wait=True)
    
    new_item = json_item(
              art_type=list(art_type.value),
              name=name_box.value,
              coordinates=marker.location,
              geo_type='Point',
              address=address_box.value,
              description=description_box.value,
              artist=artist_box.value,
              hazards=list(warnings_select.value),
              approved=False,)

    
    
    library.append(new_item)
    
    save_library(library)
    
    with out:
        display(saved_message(new_item))
    
marker_data = {}

def display_results(x):

    
    out.clear_output(wait=True)
    #map_result = create_map(address=address_box.value,
      #                      mode=method.value)
    
    center = address_to_coord(address_box.value)
    
    m = ipl.Map(center=center, zoom=16)

    global marker
    marker= ipl.Marker(location=center, draggable=True)
    
    marker_data['location'] = marker.location

    m.add_layer(marker)

    m
    
    
    with out:
        display(m,add_box)
        

submit = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Submit',
    icon=''
)


name_box = widgets.Text(
    value='',
    placeholder='Name the Artwork',
    description='Name:',
    layout=widgets.Layout(width='auto'),
    disabled=False
)


address_box = widgets.Text(
    value='1 Courthouse Square, Graham, NC, 27253',
    placeholder='1 Courthouse Square, Graham, NC, 27253',
    description='Address:',
    layout=widgets.Layout(width='auto'),
    disabled=False
)

art_type = widgets.SelectMultiple(
    options=sorted(collection['metadata']['art_types']),
    value=['Sculpture'],
    rows=6,
    description='Type of Art',
    disabled=False
)

description_box = widgets.Textarea(
    value='',
    placeholder='Give a good description of what you are submitting. Give instructions on how to find it if the work is not visible from the street. Indicate operating hours if the location is gated.',
    description='Description:',
    layout=widgets.Layout(width='auto'),
    disabled=False
)

artist_box = widgets.Text(
    value='Unknown',
    placeholder='Name of the artist.',
    description='Artist:',
    layout=widgets.Layout(width='auto'),
    disabled=False
)

warnings_select = widgets.SelectMultiple(
    options=sorted(collection['metadata']['warnings']),
    value=['None'],
    rows=6,
    description='Type of Art',
    disabled=False
)

save = widgets.Button(
    description='Save',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Submit',
    icon=''
)


initial = [address_box, submit]
add_item = [name_box, address_box, art_type, description_box, artist_box, warnings_select, save]

box_layout = widgets.Layout(display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            border='dash',
                            width='70%')

box = widgets.Box(children=initial, layout=box_layout)
add_box = widgets.Box(children=add_item, layout=box_layout)


out = widgets.Output()

with out:
    display(box)
    


submit.on_click(display_results)    

save.on_click(save_new_item)

out

Output()

In [4]:
list(art_type.value)

['Sculpture']